In [0]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import *
from datetime import datetime, timedelta

print("=== ANÁLISE DE CREATORS ===")
print(f"Executado em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

users_yt = spark.table("default.users_yt")
silver_posts_creator = spark.table("default.silver_post_creator")

print(f"Users YT: {users_yt.count()} registros")
print(f"Silver Posts Creator: {silver_posts_creator.count()} registros")

joined_data = silver_posts_creator.join(
    users_yt,
    silver_posts_creator.yt_user == users_yt.user_id,
    "left"
)

print(f"Dados após join: {joined_data.count()} registros")

=== ANÁLISE DE CREATORS ===
Executado em: 2025-08-18 22:53:56
Users YT: 10 registros
Silver Posts Creator: 12045 registros
Dados após join: 12045 registros


## Top 3 Posts por Likes (Últimos 2 Anos)
Observação: O enunciado pede posts dos últimos seis meses, mas creio que a extração dos dados foi anterior a isso, por não há posts no último semestre. Aumentei para dois anos.

In [0]:
# Abaixo está o código Python, mas o Databricks Free Edition estava
# apresentando problema para conectar, então também fiz em SQL, considerando
# que o servidor é outro. 
# (Serverless Starter Warehouse (SQL) vs. Serverless (Python)).
#
# SELECT user_id, title, likes, rank
# FROM (
#   SELECT 
#     u.user_id,
#     s.title,
#     s.likes,
#     ROW_NUMBER() OVER (PARTITION BY u.user_id ORDER BY s.likes DESC) as rank
#   FROM default.silver_post_creator s
#   INNER JOIN default.users_yt u ON s.yt_user = u.user_id
#   WHERE s.published_at >= DATE_SUB(current_date(), 730)
# )
# WHERE rank <= 3
# ORDER BY user_id, rank;

data_limite = (datetime.now() - timedelta(days=730))
print(f"Data limite para análise (últimos 6 meses): {data_limite}")

recent_posts_likes = joined_data.filter(
    (F.col("published_at") >= data_limite) & 
    (F.col("user_id").isNotNull())
)

window_likes = Window.partitionBy("user_id").orderBy(F.col("likes").desc())

top_posts_likes = recent_posts_likes.withColumn(
    "rank", F.row_number().over(window_likes)
).filter(
    F.col("rank") <= 3
).select(
    "user_id", "title", "likes", "rank"
).orderBy("user_id", "rank")

print("=== TOP 3 POSTS POR LIKES (ÚLTIMOS 6 MESES) ===")
top_posts_likes.show(50, truncate=False)

Data limite para análise (últimos 6 meses): 2023-08-19 22:53:58.948668
=== TOP 3 POSTS POR LIKES (ÚLTIMOS 6 MESES) ===
+--------------+-----------------------------------------------------------------------------------------+-------+----+
|user_id       |title                                                                                    |likes  |rank|
+--------------+-----------------------------------------------------------------------------------------+-------+----+
|PewDiePie     |I Drew Every Day for 100 DAYS!                                                           |531425 |1   |
|PewDiePie     |Teaching myself to draw for no reason                                                    |354677 |2   |
|PewDiePie     |This is the best                                                                         |276399 |3   |
|Pirulla25     |Os alienígenas apresentados no México (#Pirula 375)                                      |30035  |1   |
|Pirulla25     |FÓSSIL REVELA QUE O SER H

  File <command-6162900690643732>, line 3
    WITH recent_posts AS (
         ^
SyntaxError: invalid syntax


## Top 3 Posts por Views (Últimos 2 Anos)

In [0]:
# SELECT user_id, title, views, rank
# FROM (
#   SELECT 
#     u.user_id,
#     s.title,
#     s.views,
#     ROW_NUMBER() OVER (PARTITION BY u.user_id ORDER BY s.views DESC) as rank
#   FROM default.silver_post_creator s
#   INNER JOIN default.users_yt u ON s.yt_user = u.user_id
#   WHERE s.published_at >= DATE_SUB(current_date(), 730)
# )
# WHERE rank <= 3
# ORDER BY user_id, rank;

recent_posts_views = joined_data.filter(
    (F.col("published_at") >= data_limite) & 
    (F.col("user_id").isNotNull())
)

window_views = Window.partitionBy("user_id").orderBy(F.col("views").desc())

top_posts_views = recent_posts_views.withColumn(
    "rank", F.row_number().over(window_views)
).filter(
    F.col("rank") <= 3
).select(
    "user_id", "title", "views", "rank"
).orderBy("user_id", "rank")

print("=== TOP 3 POSTS POR VIEWS (ÚLTIMOS 6 MESES) ===")
top_posts_views.show(50, truncate=False)

=== TOP 3 POSTS POR VIEWS (ÚLTIMOS 6 MESES) ===
+--------------+-----------------------------------------------------------------------------------------+---------+----+
|user_id       |title                                                                                    |views    |rank|
+--------------+-----------------------------------------------------------------------------------------+---------+----+
|PewDiePie     |I Drew Every Day for 100 DAYS!                                                           |5470620  |1   |
|PewDiePie     |The real reason I left Sweden.                                                           |4765068  |2   |
|PewDiePie     |dad life                                                                                 |4087996  |3   |
|Pirulla25     |Os alienígenas apresentados no México (#Pirula 375)                                      |306514   |1   |
|Pirulla25     |FÓSSIL REVELA QUE O SER HUMANO SURGIU NA EUROPA? CUIDADO, NÃO É NADA DISSO! (#Piru

## Creators Órfãos

In [0]:
# SELECT 
#   s.yt_user,
#   COUNT(*) as post_count
# FROM default.silver_post_creator s
# LEFT ANTI JOIN default.users_yt u ON s.yt_user = u.user_id
# WHERE s.yt_user IS NOT NULL
# GROUP BY s.yt_user
# ORDER BY s.yt_user;

orphan_users = silver_posts_creator.select("yt_user").distinct().join(
    users_yt.select("user_id"),
    silver_posts_creator.yt_user == users_yt.user_id,
    "left_anti"
).filter(
    F.col("yt_user").isNotNull()
)

orphan_with_counts = orphan_users.join(
    silver_posts_creator.groupBy("yt_user").count().withColumnRenamed("count", "post_count"),
    "yt_user"
).orderBy("yt_user")

print("=== CREATORS ÓRFÃOS ===")
print("'yt_users' que estão na tabela silver_posts_creator mas não estão na tabela users_yt:")
orphan_with_counts.show(50, truncate=False)

=== CREATORS ÓRFÃOS ===
'yt_users' que estão na tabela silver_posts_creator mas não estão na tabela users_yt:
+-----------------+----------+
|yt_user          |post_count|
+-----------------+----------+
|CanalKondZilla   |334       |
|DottieChicken    |1         |
|GallinaPintadita |2         |
|LuisitoComunica  |127       |
|MoreZoella       |43        |
|checkgate        |248       |
|raywilliamjohnson|646       |
|tedtalksdirector |326       |
|tedx Talks       |9041      |
+-----------------+----------+



## Publicações por Mês de Cada Creator

In [0]:
# SELECT 
#   user_id,
#   SUM(post_count) as total_posts,
#   AVG(post_count) as avg_monthly_posts,
#   MAX(post_count) as max_monthly_posts
# FROM (
#   SELECT 
#     u.user_id,
#     DATE_FORMAT(s.published_at, 'yyyy-MM') as year_month,
#     COUNT(*) as post_count
#   FROM default.silver_post_creator s
#   INNER JOIN default.users_yt u ON s.yt_user = u.user_id
#   GROUP BY u.user_id, DATE_FORMAT(s.published_at, 'yyyy-MM')
# )
# GROUP BY user_id
# ORDER BY total_posts DESC;

monthly_data = joined_data.filter(
    F.col("user_id").isNotNull()
).withColumn(
    "year_month", F.date_format(F.col("published_at"), "yyyy-MM")
)

monthly_counts = monthly_data.groupBy("user_id", "year_month").count()
all_months = monthly_data.select("year_month").distinct()
all_users = monthly_data.select("user_id").distinct()
all_combinations = all_users.crossJoin(all_months)
complete_monthly_data = all_combinations.join(
    monthly_counts,
    ["user_id", "year_month"],
    "left"
).fillna(0, subset=["count"])

print("=== PUBLICAÇÕES POR MÊS ===")
print("Resumo de publicações mensais por creator:")

monthly_summary = complete_monthly_data.groupBy("user_id").agg(
    F.sum("count").alias("total_posts"),
    F.avg("count").alias("media_mensal"),
    F.max("count").alias("max_mensal")
).orderBy("total_posts", ascending=False)

monthly_summary.show(20)

=== PUBLICAÇÕES POR MÊS ===
Resumo de publicações mensais por creator:
+--------------+-----------+------------------+----------+
|       user_id|total_posts|      media_mensal|max_mensal|
+--------------+-----------+------------------+----------+
|    luccasneto|        492| 37.84615384615385|        56|
|    felipeneto|        374| 28.76923076923077|        71|
|portadosfundos|        312|              24.0|        28|
|     Pirulla25|         66| 5.076923076923077|         8|
|     PewDiePie|         33|2.5384615384615383|         4|
+--------------+-----------+------------------+----------+



## Publicações por Mês

In [0]:
complete_monthly_pandas = complete_monthly_data.toPandas()

pivot_table = complete_monthly_pandas.pivot(
    index='user_id', 
    columns='year_month', 
    values='count'
).fillna(0).astype(int)

print("=== PUBLICAÇÕES POR MÊS ===")
print(f"Tabela pivot: {len(pivot_table)} creators x {len(pivot_table.columns)} meses")
print("\nTabela pivot completa (user_id nas linhas, meses nas colunas):")

pivot_spark_df = spark.createDataFrame(
    pivot_table.reset_index().rename_axis(None, axis=1)
)

pivot_spark_df.show(20, truncate=False)

=== PUBLICAÇÕES POR MÊS ===
Tabela pivot: 5 creators x 13 meses

Tabela pivot completa (user_id nas linhas, meses nas colunas):
+--------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
|user_id       |2023-04|2023-05|2023-06|2023-07|2023-08|2023-09|2023-10|2023-11|2023-12|2024-01|2024-02|2024-03|2024-04|
+--------------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+
|PewDiePie     |1      |3      |3      |0      |2      |4      |3      |4      |3      |2      |3      |4      |1      |
|Pirulla25     |5      |3      |5      |7      |5      |7      |4      |5      |8      |6      |3      |6      |2      |
|felipeneto    |18     |39     |30     |35     |39     |28     |22     |25     |21     |0      |71     |36     |10     |
|luccasneto    |27     |56     |44     |48     |28     |32     |18     |45     |46     |45     |37     |43     |23     |
|portadosfundos|12     |2